In [ ]:
# Import necessary libraries
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define the path to your dataset
dataset_path = '/content/drive/My Drive/Data Sets/Training-Data/lettuce'


In [ ]:
# Define the number of classes
num_classes = 5

In [ ]:
# Data preprocessing and augmentation
datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, validation_split=0.2)


In [ ]:
# Generate training dataset
train_generator = datagen.flow_from_directory(dataset_path, target_size=(224, 224), batch_size=32, class_mode='categorical', subset='training')


Found 30 images belonging to 5 classes.


In [ ]:
# Generate validation dataset
val_generator = datagen.flow_from_directory(dataset_path, target_size=(224, 224), batch_size=32, class_mode='categorical', subset='validation')


Found 5 images belonging to 5 classes.


In [ ]:
# Load pre-trained MobileNetV2 without top classification layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False


In [ ]:
# Create a new model on top
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(train_generator, validation_data=val_generator, epochs=10)

Epoch 1/10
1/1 [==============================] - 10s 10s/step - loss: 2.1300 - accuracy: 0.2000 - val_loss: 1.6477 - val_accuracy: 0.4000
Epoch 2/10
1/1 [==============================] - 2s 2s/step - loss: 1.4344 - accuracy: 0.4667 - val_loss: 1.4286 - val_accuracy: 0.4000
Epoch 3/10
1/1 [==============================] - 2s 2s/step - loss: 1.0131 - accuracy: 0.7333 - val_loss: 1.2279 - val_accuracy: 0.6000
Epoch 4/10
1/1 [==============================] - 3s 3s/step - loss: 0.8894 - accuracy: 0.7333 - val_loss: 1.1114 - val_accuracy: 0.6000
Epoch 5/10
1/1 [==============================] - 2s 2s/step - loss: 0.7313 - accuracy: 0.8000 - val_loss: 0.6421 - val_accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 2s 2s/step - loss: 0.5551 - accuracy: 0.8667 - val_loss: 0.4430 - val_accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 2s 2s/step - loss: 0.4800 - accuracy: 0.8667 - val_loss: 0.4286 - val_accuracy: 1.0000
Epoch 8/10
1/1 [=======================

In [ ]:
# Save the trained model
model.save('trained_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
def preprocess_image(img_path):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalize pixel values
    return img_array

In [ ]:
def predict_lettuce_type(img_path, confidence_threshold=0.1):
    img_array = preprocess_image(img_path)
    predictions = model.predict(img_array)
    class_index = np.argmax(predictions)
    confidence = predictions[0, class_index]

    if confidence >= confidence_threshold:
        if class_index == 0:
            return "Butterhead"
        elif class_index == 1:
            return "Green Baatvia"
        elif class_index == 2:
            return "Iceberg"
        elif class_index == 3:
            return "Lollo Rosso"
        elif class_index == 4:
            return "Romaine"

    else:
        return "Unknown"

In [ ]:
!pip install dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 18.5 MB/s eta 0:00:00


In [ ]:
from dash import Dash, dcc, html, Input, Output, State
from dash.exceptions import PreventUpdate
import base64

In [ ]:
app = Dash(__name__)

In [ ]:
app.layout = html.Div([
    html.H1("Lettuce Type Prediction"),
    dcc.Upload(
        id='upload-image',
        children=html.Div([
            'Drag and Drop or ',
            html.A('Select Files')
        ]),
        multiple=False
    ),
    html.Div(id='output-image-upload'),
])

In [ ]:
@app.callback(
    Output('output-image-upload', 'children'),
    [Input('upload-image', 'contents')],
    [State('upload-image', 'filename')],
)
def update_output(contents, filename):
    if contents is None:
        raise PreventUpdate

    try:
        # Save the uploaded image
        img_data = contents.split(',')[1]
        img_data_decoded = base64.b64decode(img_data)
        img_path = 'uploaded_image.jpg'
        with open(img_path, 'wb') as f:
            f.write(img_data_decoded)

        # Get the predicted lettuce type
        lettuce_type = predict_lettuce_type(img_path)

        # Display the result
        return [
            html.H5(f'Uploaded Image: {filename}'),
            html.Img(src=contents),
            html.H6(f'Predicted Lettuce Type: {lettuce_type}')
        ]

    except Exception as e:
        return f'Error: {str(e)}'

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>